# NLP Lab4 - Language modelling

**Author: Bartłomiej Jamiołkowski**

The exercise shows how a language model may be used to solve word-prediction tasks and to generate text.

In [1]:
from transformers import pipeline
from termcolor import colored
import pandas as pd
import itertools
import warnings
warnings.filterwarnings('ignore')

## Tasks 1 - 2

Read the documentation of [Language modelling in the Transformers](https://huggingface.co/transformers/task_summary.html#language-modeling) library.

Download three [Polish models](https://huggingface.co/models?filter=pl) from the Huggingface repository. These should be regular language models, which were not fine-tuned. E.g. `HerBERT` and `papuGaPT2` are good examples. You can also try using Bielik for that, but make sure you are using the model via Transformers API, not GUI.

In [3]:
twhin_bert_fill_mask = pipeline(task = 'fill-mask', model = 'Twitter/twhin-bert-base')
herbert_base_fill_mask = pipeline(task = 'fill-mask', model = 'allegro/herbert-base-cased')
xlm_roberta_fill_mask = pipeline(task = 'fill-mask', model = 'FacebookAI/xlm-roberta-base')

Some weights of the model checkpoint at FacebookAI/xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Task 3

Devise a method to test if the langage model understands Polish cases. E.g. testing for *nominal case* could be expressed as "Warszawa to największe `[MASK]`", and the masked word should be in nominative case. Create sentences for each case.

In [5]:
polish_cases_sentences_dict = {
    'nominative': 'Warszawa to największe <mask>.',
    'genitive': 'Brakuje mi smaku słodkiego <mask> w mojej herbacie.',
    'dative': 'Daję napiwek uprzejmemu <mask> w restauracji.',
    'accusative': 'Zbudowałem mały <mask> na plaży.',
    'instrumental': 'Piszę nowym <mask>, który wczoraj kupiłem.',
    'locative': 'Czytałem o ciekawym <mask> w gazecie.',
    'vocative': 'Mądry <mask>, wysłuchaj moich słów!'
}

In [6]:
def test_models_topic_understanding(model_list, sentences_dict: dict, k: int) -> None:
    for model in model_list:
        print(f"{'-' * 45}\nModel: {model.model.config._name_or_path.split('/')[1]}\n{'-' * 45}")

        model_results = {}
    
        for topic, sentence in sentences_dict.items():
            print(f'\n{topic.capitalize()}:')
        
            preds = model(sentence, top_k = k)

            words_dict = {f'word_{idx + 1}': pred['token_str'] for idx, pred in enumerate(preds)}
            scores_dict = {f'score_{idx + 1}': pred['score'] for idx, pred in enumerate(preds)}
            model_results[topic] = dict(itertools.chain(*zip(words_dict.items(), scores_dict.items())))
        
            print(sentence.replace('<mask>', colored('<mask>', 'red')))
            print(preds[0]['sequence'].replace(preds[0]['token_str'], colored(preds[0]['token_str'], 'green')))
        
        display(pd.DataFrame.from_dict(model_results, orient = 'index'))

In [7]:
model_list = [twhin_bert_fill_mask, herbert_base_fill_mask, xlm_roberta_fill_mask]

In [10]:
test_models_topic_understanding(model_list, polish_cases_sentences_dict, k = 3)

---------------------------------------------
Model: twhin-bert-base
---------------------------------------------

Nominative:
Warszawa to największe <mask>.
Warszawa to największe miasto .

Genitive:
Brakuje mi smaku słodkiego <mask> w mojej herbacie.
Brakuje mi smaku słodkiego, w mojej herbacie.

Dative:
Daję napiwek uprzejmemu <mask> w restauracji.
Daję napiwek uprzejmemu dzisiaj w restauracji.

Accusative:
Zbudowałem mały <mask> na plaży.
Zbudowałem mały dom na plaży.

Instrumental:
Piszę nowym <mask>, który wczoraj kupiłem.
Piszę nowym album , który wczoraj kupiłem.

Locative:
Czytałem o ciekawym <mask> w gazecie.
Czytałem o ciekawym rynku w gazecie.

Vocative:
Mądry <mask>, wysłuchaj moich słów!
Mądry Boże , wysłuchaj moich słów!


,word_1,score_1,word_2,score_2,word_3,score_3
nominative,miasto,0.736737,miasta,0.054660,miejsce,0.032278
genitive,",",0.074094,kota,0.048996,mleka,0.037487
dative,dzisiaj,0.068017,dziś,0.054846,jak,0.046457
accusative,dom,0.146561,stadion,0.052840,samochód,0.025140
instrumental,album,0.310780,książkę,0.044605,telefon,0.036752
locative,rynku,0.283439,świecie,0.071885,portalu,0.041350
vocative,Boże,0.247848,człowiek,0.055606,Putin,0.015251


---------------------------------------------
Model: herbert-base-cased
---------------------------------------------

Nominative:
Warszawa to największe <mask>.
Warszawa to największe miasto .

Genitive:
Brakuje mi smaku słodkiego <mask> w mojej herbacie.
Brakuje mi smaku słodkiego mleka w mojej herbacie .

Dative:
Daję napiwek uprzejmemu <mask> w restauracji.
Daję napiwek uprzejmemu klientowi w restauracji .

Accusative:
Zbudowałem mały <mask> na plaży.
Zbudowałem mały domek na plaży .

Instrumental:
Piszę nowym <mask>, który wczoraj kupiłem.
Piszę nowym tekstem , który wczoraj kupiłem .

Locative:
Czytałem o ciekawym <mask> w gazecie.
Czytałem o ciekawym artykule w gazecie .

Vocative:
Mądry <mask>, wysłuchaj moich słów!
Mądry człowieku , wysłuchaj moich słów !


,word_1,score_1,word_2,score_2,word_3,score_3
nominative,miasto,0.810391,lotnisko,0.082490,centrum,0.026521
genitive,mleka,0.133326,cukru,0.128373,miodu,0.119054
dative,klientowi,0.307106,koledze,0.224259,panu,0.175446
accusative,domek,0.589590,ogródek,0.105919,ogród,0.047390
instrumental,tekstem,0.315495,artykułem,0.158764,artykule,0.030858
locative,artykule,0.637309,temacie,0.069565,miejscu,0.045275
vocative,człowieku,0.529148,Polak,0.079474,jesteś,0.060546


---------------------------------------------
Model: xlm-roberta-base
---------------------------------------------

Nominative:
Warszawa to największe <mask>.
Warszawa to największe miasto .

Genitive:
Brakuje mi smaku słodkiego <mask> w mojej herbacie.
Brakuje mi smaku słodkiego cukru w mojej herbacie.

Dative:
Daję napiwek uprzejmemu <mask> w restauracji.
Daję napiwek uprzejmemu, w restauracji.

Accusative:
Zbudowałem mały <mask> na plaży.
Zbudowałem mały dom na plaży.

Instrumental:
Piszę nowym <mask>, który wczoraj kupiłem.
Piszę nowym blogu , który wczoraj kupiłem.

Locative:
Czytałem o ciekawym <mask> w gazecie.
Czytałem o ciekawym artykule w gazecie.

Vocative:
Mądry <mask>, wysłuchaj moich słów!
Mądry Boże , wysłuchaj moich słów!


,word_1,score_1,word_2,score_2,word_3,score_3
nominative,miasto,0.525982,miasta,0.050481,,0.023420
genitive,cukru,0.271794,alkoholu,0.165072,mleka,0.074708
dative,",",0.132466,i,0.101560,gości,0.072060
accusative,dom,0.627250,sklep,0.075990,hotel,0.059166
instrumental,blogu,0.160931,telefonu,0.097599,samochodem,0.075135
locative,artykule,0.405312,spotkaniu,0.164633,konkursie,0.084068
vocative,Boże,0.296490,człowiek,0.067605,mój,0.043820


I am going to analyse model understanding of Polish cases one by one:
- **nominative** - All tested models correctly completed the nominative case with 'miasto',
- **genitive** - the best results were obtained for 'mleko' with twhin-bert-base and 'cukru' with xlm-roberta-base. The herbert-base-cased model found ',' indicating the lack of a matching word,
- **dative** - all models failed to find an appropriate matching word,
- **accusative** - the xlm-roberta-base model performed best in the accusative, correctly selecting 'dom'. The herbert-base-cased model also had sensible answer 'domek', but with slightly worse fit,
- **instrumental** - all models failed to find an appropriate matching word,
- **locative** - all models found matching words, but I would not say these matchings were the best,
- **vocative** - the herbert-base-cased model achieved the best result with the answer 'człowieku'. Other models returned not such obvious results but still acceptable

Summarising results of an analysis, tested models do not understand all Polish cases. The biggest problems occured in : genitive, dative and instrumental. In contrast all models searched nominative quite well (even second word options were mostly close to an appropriate word).

## Task 4

Devise a method to test long-range relationships such as gender. E.e. you can use two verbs with masculine and feminine gender, where one of the verbs is masked. Both verbs should have the same gender, assuming the subject is the same. Define at least 3 such sentences.

In [48]:
gender_sentences_dict = {
    'masculine_1': 'Mężczyzna wziął arbuz do ręki i <mask> go na kawałki.',
    'masculine_2': 'On najpierw zbudował model statku, a następnie <mask> mu zdjęcie.',
    'masculine_3': 'Programista <mask> nowy program, po czym pochwalił się nim w internecie.',
    'feminine_1': 'Dziewczyna weszła do piekarni i <mask> bułki na śniadanie.',
    'feminine_2': 'Ona kupiła gorącą kawę, a następnie ją <mask>.',
    'feminine_3': 'Siostra zapakowała prezent w kolorowy papier i <mask> go do torby.'
}

In [49]:
test_models_topic_understanding(model_list, gender_sentences_dict, k = 3)

---------------------------------------------
Model: twhin-bert-base
---------------------------------------------

Masculine_1:
Mężczyzna wziął arbuz do ręki i <mask> go na kawałki.
Mężczyzna wziął arbuz do ręki i trafił go na kawałki.

Masculine_2:
On najpierw zbudował model statku, a następnie <mask> mu zdjęcie.
On najpierw zbudował model statku, a następnie zrobił mu zdjęcie.

Masculine_3:
Programista <mask> nowy program, po czym pochwalił się nim w internecie.
Programista stworzył nowy program, po czym pochwalił się nim w internecie.

Feminine_1:
Dziewczyna weszła do piekarni i <mask> bułki na śniadanie.
Dziewczyna weszła do piekarni i kupił bułki na śniadanie.

Feminine_2:
Ona kupiła gorącą kawę, a następnie ją <mask>.
Ona kupiła gorącą kawę, a następnie ją kupił .

Feminine_3:
Siostra zapakowała prezent w kolorowy papier i <mask> go do torby.
Siostra zapakowała prezent w kolorowy papier i kupił go do torby.


,word_1,score_1,word_2,score_2,word_3,score_3
masculine_1,trafił,0.165227,zmienił,0.097366,wpadł,0.071723
masculine_2,zrobił,0.699268,pokazał,0.084741,kupił,0.035828
masculine_3,stworzył,0.220869,miał,0.139517,zrobił,0.107221
feminine_1,kupił,0.244010,robi,0.169651,zrobił,0.067073
feminine_2,kupił,0.142151,pije,0.050519,zaczęła,0.030119
feminine_3,kupił,0.083535,dostala,0.074910,chciała,0.056530


---------------------------------------------
Model: herbert-base-cased
---------------------------------------------

Masculine_1:
Mężczyzna wziął arbuz do ręki i <mask> go na kawałki.
Mężczyzna wziął arbuz do ręki i podzielił go na kawałki .

Masculine_2:
On najpierw zbudował model statku, a następnie <mask> mu zdjęcie.
On najpierw zbudował model statku , a następnie zrobił mu zdjęcie .

Masculine_3:
Programista <mask> nowy program, po czym pochwalił się nim w internecie.
Programista stworzył nowy program , po czym pochwalił się nim w internecie .

Feminine_1:
Dziewczyna weszła do piekarni i <mask> bułki na śniadanie.
Dziewczyna weszła do piekarni i przyniosła bułki na śniadanie .

Feminine_2:
Ona kupiła gorącą kawę, a następnie ją <mask>.
Ona kupiła gorącą kawę , a następnie ją przyniosła .

Feminine_3:
Siostra zapakowała prezent w kolorowy papier i <mask> go do torby.
Siostra zapakowała prezent w kolorowy papier i przyniosła go do torby .


,word_1,score_1,word_2,score_2,word_3,score_3
masculine_1,podzielił,0.385333,rozbił,0.129012,złamał,0.039373
masculine_2,zrobił,0.773811,wykonał,0.102137,robił,0.032378
masculine_3,stworzył,0.307816,opracował,0.113065,przygotował,0.095846
feminine_1,przyniosła,0.676425,kupiła,0.117508,zrobiła,0.032191
feminine_2,przyniosła,0.209883,kupiła,0.103914,podała,0.072724
feminine_3,przyniosła,0.196058,zabrała,0.190756,wniosła,0.083803


---------------------------------------------
Model: xlm-roberta-base
---------------------------------------------

Masculine_1:
Mężczyzna wziął arbuz do ręki i <mask> go na kawałki.
Mężczyzna wziął arbuz do ręki i zrobił go na kawałki.

Masculine_2:
On najpierw zbudował model statku, a następnie <mask> mu zdjęcie.
On najpierw zbudował model statku, a następnie zrobił mu zdjęcie.

Masculine_3:
Programista <mask> nowy program, po czym pochwalił się nim w internecie.
Programista stworzył nowy program, po czym pochwalił się nim w internecie.

Feminine_1:
Dziewczyna weszła do piekarni i <mask> bułki na śniadanie.
Dziewczyna weszła do piekarni i dala bułki na śniadanie.

Feminine_2:
Ona kupiła gorącą kawę, a następnie ją <mask>.
Ona kupiła gorącą kawę, a następnie ją pije .

Feminine_3:
Siostra zapakowała prezent w kolorowy papier i <mask> go do torby.
Siostra zapakowała prezent w kolorowy papier i dala go do torby.


,word_1,score_1,word_2,score_2,word_3,score_3
masculine_1,zrobił,0.298887,dal,0.185793,ł,0.041491
masculine_2,zrobił,0.703186,pokazał,0.129618,stworzył,0.036255
masculine_3,stworzył,0.551055,napisał,0.179630,rozpoczął,0.048421
feminine_1,dala,0.319552,pani,0.224230,robi,0.074098
feminine_2,pije,0.392522,spala,0.080102,kupił,0.037110
feminine_3,dala,0.921451,doda,0.016736,dodaje,0.009135


I am going to analyse model one by one in order to check if they understand a long-range relationships such as gender:
- **masculine_1** - all models suggested masculine words. However only herbert-base-cased found an appropriate word for the sentence.
- **masculine_2** - all models proper masculine words. The second found word returned by herbert-base-cased model was also good option for this sentence,
- **masculine_3** - all models returned appropaite masculine words in the rank,
- **feminine_1** - only one model herbert-base-cased managed to return fittiing feminine word. A xlm-roberta-base even returned unknown word 'dala'.
- **feminine_2** - in my opion only herbert-base-cased model suggested fitting word 'przyniosła'. Other models failed, particularly twhin-bert-base , which returned 'kupił' for feminine word,
- **feminine_3** - the only successfull model in this case was herbert-base-cased, which suggested przyniosła (it is not top option , but I accept it). Other models failed inluding xlm-roberta-base model, which returned unknown word 'dala'.

Having insight into obtained results I can conclude that the herbert-base-cased model showed the best overall understanding of gender in sentences. This model correctly handled both masculine and feminine contexts. In contrast, xlm-roberta-base and twhin-bert-base models faced issues, particularly in feminine contexts, failing to return correct gendered words in several instances.

## Task 5

Check if the model captures real-world knolwedge. For instance a sentence "`[MASK]` wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza." checks if the model "knows" the description of water. Define at least 3 such sentences.

In [12]:
real_world_knowledge_dict = {
    'sentence_1': 'Ziemia jest trzecią <mask> w Układzie Słonecznym.',
    'sentence_2': 'Woda składa się z dwóch atomów wodoru i jednego atomu <mask>.',
    'sentence_3': 'Największym kontynentem na świecie jest <mask>.',
    'sentence_4': 'Największą prędkość we Wszechświecie osiąga <mask>.',
    'sentence_5': 'Mount Everest jest najwyższą <mask> na Ziemi.'
}

In [13]:
test_models_topic_understanding(model_list, real_world_knowledge_dict, k = 3)

---------------------------------------------
Model: twhin-bert-base
---------------------------------------------

Sentence_1:
Ziemia jest trzecią <mask> w Układzie Słonecznym.
Ziemia jest trzecią osobą w Układzie Słonecznym.

Sentence_2:
Woda składa się z dwóch atomów wodoru i jednego atomu <mask>.
Woda składa się z dwóch atomów wodoru i jednego atomu putin .

Sentence_3:
Największym kontynentem na świecie jest <mask>.
Największym kontynentem na świecie jest Polska .

Sentence_4:
Największą prędkość we Wszechświecie osiąga <mask>.
Największą prędkość we Wszechświecie osiąga Bóg .

Sentence_5:
Mount Everest jest najwyższą <mask> na Ziemi.
Mount Everest jest najwyższą osobą na Ziemi.


,word_1,score_1,word_2,score_2,word_3,score_3
sentence_1,osobą,0.791013,miejsce,0.039414,ręką,0.015163
sentence_2,putin,0.108221,atom,0.076849,NATO,0.052713
sentence_3,Polska,0.101999,Twitter,0.079794,Putin,0.075968
sentence_4,Bóg,0.184124,Polska,0.057120,my,0.035603
sentence_5,osobą,0.647363,częścią,0.053028,wodą,0.018888


---------------------------------------------
Model: herbert-base-cased
---------------------------------------------

Sentence_1:
Ziemia jest trzecią <mask> w Układzie Słonecznym.
Ziemia jest trzecią gwiazdą w Układzie Słonecznym .

Sentence_2:
Woda składa się z dwóch atomów wodoru i jednego atomu <mask>.
Woda składa się z dwóch atomów wodoru i jednego atomu tlenu .

Sentence_3:
Największym kontynentem na świecie jest <mask>.
Największym kontynentem na świecie jest Japonia .

Sentence_4:
Największą prędkość we Wszechświecie osiąga <mask>.
Największą prędkość we Wszechświecie osiąga Słońce .

Sentence_5:
Mount Everest jest najwyższą <mask> na Ziemi.
Mount Everest jest najwyższą górą na Ziemi .


,word_1,score_1,word_2,score_2,word_3,score_3
sentence_1,gwiazdą,0.290387,planeta,0.220715,istotą,0.181981
sentence_2,tlenu,0.383143,węgla,0.164454,siarki,0.151711
sentence_3,Japonia,0.226761,Australia,0.157105,Brazylia,0.088474
sentence_4,Słońce,0.346972,Księżyc,0.116756,Ziemia,0.074622
sentence_5,górą,0.835898,góra,0.042872,kulą,0.013952


---------------------------------------------
Model: xlm-roberta-base
---------------------------------------------

Sentence_1:
Ziemia jest trzecią <mask> w Układzie Słonecznym.
Ziemia jest trzecią osobą w Układzie Słonecznym.

Sentence_2:
Woda składa się z dwóch atomów wodoru i jednego atomu <mask>.
Woda składa się z dwóch atomów wodoru i jednego atomu wody .

Sentence_3:
Największym kontynentem na świecie jest <mask>.
Największym kontynentem na świecie jest Kuba .

Sentence_4:
Największą prędkość we Wszechświecie osiąga <mask>.
Największą prędkość we Wszechświecie osiągał .

Sentence_5:
Mount Everest jest najwyższą <mask> na Ziemi.
Mount Everest jest najwyższą wysokości na Ziemi.


,word_1,score_1,word_2,score_2,word_3,score_3
sentence_1,osobą,0.309851,kartą,0.288977,częścią,0.135639
sentence_2,wody,0.390897,soli,0.332214,CO,0.034282
sentence_3,Kuba,0.278558,Madagaskar,0.082982,India,0.036254
sentence_4,ł,0.549325,ła,0.047669,ły,0.044128
sentence_5,wysokości,0.331644,wodą,0.227669,częścią,0.079105


I am going to analyse whether given models are capable of capturing real-world knolwedge or not:
- **sentence_1** - all models identified wrong names demonstrating a lack of astronomical knowledge,
- **sentence_2** - only herbert-base-cased model proved having some kind of knowledge by suggesting 'tlen' as a suitable word,
- **sentence_3** - all models identified wrong words demonstrating a lack of geographical knowledge,
- **sentence_4** - all models identified wrong words demonstrating a lack of knowledge about Physics,
- **sentence_5** - only herbert-base-cased model proved having some knowledge about geography. Other models suggested wrong answers,

Obtained results allow me to make a conclusion that choosen models do not capture real-world knolwedge or do it poorly. It has to be mentioned that herbert-base-cased model managed to retrun two good answers for five sentences. It gives me hope that mentioned model has some small capability of capturing  real-world knolwedge.

## Task 6

Check zero-shot learning capabilites of the models. Provide at least 5 sentences with different sentiment for the following scheme: "'Ten film to był kiler. Nie mogłem się oderwać od ekranu.' Wypowiedź ta ma jest zdecydowanie `[MASK]`" Try different prompts, to see if they make any difference.

In [53]:
prompts_dict = {
    'prompt_1': " Ta wypowiedź ma <mask> nacechowanie emocjonalne.",
    'prompt_2': " Wypowiedź ta jest zdecydowanie <mask>.",
    'prompt_3': " Zdanie to jest nacechowane <mask>."
}

sentiment_sentences_dict = {
    'positive_1': "'Dawno nie czytałem tak dobrej książki. Zdecydowanie polecam każdemu!'",
    'positive_2': "'Danie było idealnie przyprawione. Dobre żarcie dla każdego.'",
    'negative_1': "'Według mnie zakupienie tego produktu było zmarnowaniem pieniędzy. Co za bubel!'",
    'negative_2': "'Kelner był opryskliwy. Nie dostał napiwku.'",
}

In [54]:
for prompt_key, prompt in prompts_dict.items():
    print(f"{'-' * 45}\nPrompt: {prompt}")
    
    sentiment_sentences_copy_dict = {key: (lambda x: x + prompt)(sentence) for key, sentence in sentiment_sentences_dict.items()}
    test_models_topic_understanding(model_list, sentiment_sentences_copy_dict, k = 3)

---------------------------------------------
Prompt:  Ta wypowiedź ma <mask> nacechowanie emocjonalne.
---------------------------------------------
Model: twhin-bert-base
---------------------------------------------

Positive_1:
'Dawno nie czytałem tak dobrej książki. Zdecydowanie polecam każdemu!' Ta wypowiedź ma <mask> nacechowanie emocjonalne.
'Dawno nie czytałem tak dobrej książki. Zdecydowanie polecam każdemu!' Ta wypowiedź ma być nacechowanie emocjonalne.

Positive_2:
'Danie było idealnie przyprawione. Dobre żarcie dla każdego.' Ta wypowiedź ma <mask> nacechowanie emocjonalne.
'Danie było idealnie przyprawione. Dobre żarcie dla każdego.' Ta wypowiedź ma być nacechowanie emocjonalne.

Negative_1:
'Według mnie zakupienie tego produktu było zmarnowaniem pieniędzy. Co za bubel!' Ta wypowiedź ma <mask> nacechowanie emocjonalne.
'Według mnie zakupienie tego produktu było zmarnowaniem pieniędzy. Co za bubel!' Ta wypowiedź ma być nacechowanie emocjonalne.

Negative_2:
'Kelner był opry

,word_1,score_1,word_2,score_2,word_3,score_3
positive_1,być,0.081254,pełne,0.053161,takie,0.045235
positive_2,być,0.094567,pełne,0.070791,swoje,0.035727
negative_1,być,0.194855,pełne,0.039156,swoje,0.035181
negative_2,być,0.110883,pełne,0.047603,jakieś,0.033008


---------------------------------------------
Model: herbert-base-cased
---------------------------------------------

Positive_1:
'Dawno nie czytałem tak dobrej książki. Zdecydowanie polecam każdemu!' Ta wypowiedź ma <mask> nacechowanie emocjonalne.
' Dawno nie czytałem tak dobrej książki . Zdecydowanie polecam każdemu ! ' Ta wypowiedź ma silne nacechowanie emocjonalne .

Positive_2:
'Danie było idealnie przyprawione. Dobre żarcie dla każdego.' Ta wypowiedź ma <mask> nacechowanie emocjonalne.
' Danie było idealnie przyprawione . Dobre żarcie dla każdego . ' Ta wypowiedź ma silne nacechowanie emocjonalne .

Negative_1:
'Według mnie zakupienie tego produktu było zmarnowaniem pieniędzy. Co za bubel!' Ta wypowiedź ma <mask> nacechowanie emocjonalne.
' Według mnie zakupienie tego produktu było zmarnowaniem pieniędzy . Co za bubel ! ' Ta wypowiedź ma silne nacechowanie emocjonalne .

Negative_2:
'Kelner był opryskliwy. Nie dostał napiwku.' Ta wypowiedź ma <mask> nacechowanie emocjonalne.
' 

,word_1,score_1,word_2,score_2,word_3,score_3
positive_1,silne,0.263757,duże,0.139661,mocne,0.086557
positive_2,silne,0.265179,duże,0.142406,wyraźne,0.078462
negative_1,silne,0.274398,duże,0.084623,wyraźne,0.064167
negative_2,silne,0.240431,duże,0.091154,pewne,0.085570


---------------------------------------------
Model: xlm-roberta-base
---------------------------------------------

Positive_1:
'Dawno nie czytałem tak dobrej książki. Zdecydowanie polecam każdemu!' Ta wypowiedź ma <mask> nacechowanie emocjonalne.
'Dawno nie czytałem tak dobrej książki. Zdecydowanie polecam każdemu!' Ta wypowiedź ma ogromne nacechowanie emocjonalne.

Positive_2:
'Danie było idealnie przyprawione. Dobre żarcie dla każdego.' Ta wypowiedź ma <mask> nacechowanie emocjonalne.
'Danie było idealnie przyprawione. Dobre żarcie dla każdego.' Ta wypowiedź ma pewne nacechowanie emocjonalne.

Negative_1:
'Według mnie zakupienie tego produktu było zmarnowaniem pieniędzy. Co za bubel!' Ta wypowiedź ma <mask> nacechowanie emocjonalne.
'Według mnie zakupienie tego produktu było zmarnowaniem pieniędzy. Co za bubel!' Ta wypowiedź ma pewne nacechowanie emocjonalne.

Negative_2:
'Kelner był opryskliwy. Nie dostał napiwku.' Ta wypowiedź ma <mask> nacechowanie emocjonalne.
'Kelner był oprys

...

,word_1,score_1,word_2,score_2,word_3,score_3
positive_1,.,0.578062,...,0.101390,!,0.072242
positive_2,.,0.722980,...,0.079567,!,0.038664
negative_1,.,0.537684,!,0.098698,...,0.088908
negative_2,.,0.699597,...,0.081677,!,0.047167


---------------------------------------------
Model: herbert-base-cased
---------------------------------------------

Positive_1:
'Dawno nie czytałem tak dobrej książki. Zdecydowanie polecam każdemu!' Zdanie to jest nacechowane <mask>.
' Dawno nie czytałem tak dobrej książki . Zdecydowanie polecam każdemu ! ' Zdanie to jest nacechowane pozytywnie .

Positive_2:
'Danie było idealnie przyprawione. Dobre żarcie dla każdego.' Zdanie to jest nacechowane <mask>.
' Danie było idealnie przyprawione . Dobre żarcie dla każdego . ' Zdanie to jest nacechowane pozytywnie .

Negative_1:
'Według mnie zakupienie tego produktu było zmarnowaniem pieniędzy. Co za bubel!' Zdanie to jest nacechowane <mask>.
' Według mnie zakupienie tego produktu było zmarnowaniem pieniędzy . Co za bubel ! ' Zdanie to jest nacechowane negatywnie .

Negative_2:
'Kelner był opryskliwy. Nie dostał napiwku.' Zdanie to jest nacechowane <mask>.
' Kelner był opryskliwy . Nie dostał napiwku . ' Zdanie to jest nacechowane negatywni

,word_1,score_1,word_2,score_2,word_3,score_3
positive_1,pozytywnie,0.354650,optymizmem,0.095387,emocjami,0.086818
positive_2,pozytywnie,0.226392,smakiem,0.090595,emocjami,0.059493
negative_1,negatywnie,0.316242,pozytywnie,0.098502,emocjonalnie,0.087208
negative_2,negatywnie,0.229165,emocjonalnie,0.102099,pozytywnie,0.093584


---------------------------------------------
Model: xlm-roberta-base
---------------------------------------------

Positive_1:
'Dawno nie czytałem tak dobrej książki. Zdecydowanie polecam każdemu!' Zdanie to jest nacechowane <mask>.
'Dawno nie czytałem tak dobrej książki. Zdecydowanie polecam każdemu!' Zdanie to jest nacechowane. .

Positive_2:
'Danie było idealnie przyprawione. Dobre żarcie dla każdego.' Zdanie to jest nacechowane <mask>.
'Danie było idealnie przyprawione. Dobre żarcie dla każdego.' Zdanie to jest nacechowane. .

Negative_1:
'Według mnie zakupienie tego produktu było zmarnowaniem pieniędzy. Co za bubel!' Zdanie to jest nacechowane <mask>.
'Według mnie zakupienie tego produktu było zmarnowaniem pieniędzy. Co za bubel!' Zdanie to jest nacechowane. .

Negative_2:
'Kelner był opryskliwy. Nie dostał napiwku.' Zdanie to jest nacechowane <mask>.
'Kelner był opryskliwy. Nie dostał napiwku.' Zdanie to jest nacechowane. .


,word_1,score_1,word_2,score_2,word_3,score_3
positive_1,.,0.256097,!,0.120203,:),0.056545
positive_2,.,0.328414,przez,0.116941,wszystkim,0.027020
negative_1,.,0.338315,przez,0.086171,...,0.029276
negative_2,.,0.528764,przez,0.068308,...,0.029467


I am going to analyse zero-shot learning capabilites of the models. <br>
- **Prompt_1: Ta wypowiedź ma <mask> nacechowanie emocjonalne**  - all models failed to find positive and negative sentiment in every analysed sentence. It has to be mentioned that therbert-base-cased and xlm-roberta-base models match found words, which described an importance of the sentence insted of sentiment. Therefore, I assume the given propmpt was not interpeted by them as I was intended.
- **Prompt_2:  Wypowiedź ta jest zdecydowanie <mask>** - all models failed to find positive and negative sentiment in every analysed sentence. Insteda, they found words describing quality or credibility of a given sentence.
- **Prompt_3:  Zdanie to jest nacechowane <mask>** - only herbert-base-cased model was able to find proper sentiment for every analysed sentence. Other models failed, because they did not even returned interpretable words.

Obtained results allow me to make a conclusion that chosen prompt has a gret impact on zero-shot learning capabilites of the models. Only the last prompt allowed to obtain some sensible results. In this case only herbert-base-cased model was able to find proper sentiment for every analysed sentence. Taking into account all its results I assume that in this text it has an edge over other models.

## Task 7

Take into accout the fact, that causal language models such as PapuGaPT2 or plT5, will only generate continuations of the sentenes, so the examples have to be created according to that paradigm.

## Task 8 - Questions

Answers on these question are generally a summaries of my other conclusion made in above tasks.

## Question 1
Which of the models produced the best results?

In my answer, I am taking into account the overall results of all conducted tests. After examining my previous conclusions and printed suggestions, I would like to highlight the herbert-base-cased model, which, in my opinion, produced the best results among the considered models.

## Question 2
Was any of the models able to capture Polish grammar?

In my opinion every model was able to partially capture Polish grammar. The biggest problems occured in capturing: genitive, dative and instrumental. In contrast all models suggested nominatives quite well. Despite occured errors I would like to outline herbert-base-cased model, because its suggestions were mostly appropriate.

## Question 3
Was any of the models able to capture long-distant relationships between the words?

Having insight into obtained results I can conclude that the herbert-base-cased model was able to capture long-distant relationships between the words. This model correctly handled both masculine and feminine contexts. In contrast, xlm-roberta-base and twhin-bert-base models faced issues, particularly in feminine contexts, failing to return correct verbs in several instances.

## Question 4
Was any of the models able to capture world knowledge?

Obtained results allow me to make a conclusion that choosen models do not capture real-world knolwedge or do it poorly. It has to be mentioned that herbert-base-cased model managed to retrun two good answers for five sentences. It gives me hope that mentioned model has some small capability of capturing real-world knolwedge.

## Question 5
Was any of the models good at doing zero-shot classification?

Obtained results allow me to make a conclusion that chosen prompt has a great impact on zero-shot learning capabilites of the models. Only the last prompt allowed to obtain some sensible results. In this case only herbert-base-cased model was able to find proper sentiment for every analysed sentence. Taking into account all its results I assume that in this text it has an edge over other models.

## Question 6
What are the most striking errors made by the models?

There are a few controversial errors that I would like to mention. First of all, the models sometimes were not able to suggest any fitting words, returning ',' or ','. There were also situations where the models returned words that do not exist, e.g., 'dala.' I was disappointed when I found out that, very often, the models were not able to capture world knowledge, e.g., they returned silly answers. Finally, there were failures in detecting whether verbs should be formed as if they were done by a woman or a man.